In [4]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))
from utils import get_spike_activity, sdf, sdf_mean
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import scipy.stats as st
import pandas as pd

In [29]:
root_path = "/home/nomodel/code/NODS/results/grid_search/"
with open("/home/nomodel/code/NODS/network_configuration.json", "r") as json_file:
    net_config = json.load(json_file)

In [30]:
CS_burst_dur = net_config["devices"]["CS"]["parameters"]["burst_dur"]
CS_start_first = float(net_config["devices"]["CS"]["parameters"]["start_first"])
between_start = net_config["devices"]["CS"]["parameters"]["between_start"]
n_trials = net_config["devices"]["CS"]["parameters"]["n_trials"]
US_start_first = float(net_config["devices"]["US"]["parameters"]["start_first"])

In [31]:
cell = "pc_spikes"

grid_search = np.zeros((6,6))

for i in range(1,7):
    for j in range(1,7):

        results_path = root_path + f"min{i}_plus{1}"

        spk = get_spike_activity(cell_name=cell, path=results_path)
        sdf_mean_over_trials = []
        sdf_baseline = np.zeros((n_trials))
        sdf_cr = np.zeros((n_trials))
        for trial in range(n_trials):

            start = trial * between_start
            stop = CS_start_first + CS_burst_dur + trial * between_start

            sdf_cells = sdf(start=start, stop=stop, spk=spk, step=5)
            sdf_mean_trial = sdf_mean(sdf_cells)
            sdf_mean_over_trials.append(sdf_mean_trial)
            sdf_baseline[trial] = np.mean(sdf_mean_trial[100:150])
            sdf_cr[trial] = np.mean(sdf_mean_trial[250:300])

        sdf_change_baseline = sdf_baseline[1:] - sdf_baseline[1]
        sdf_change_cr = sdf_cr[1:] - sdf_cr[1]
        grid_search[i,j] = abs(np.mean(sdf_change_baseline[1:]) - np.mean(sdf_change_cr[1:]))

In [ ]:
cmap = plt.cm.get_cmap('coolwarm')
sns.heatmap(grid_search, cmap = cmap)
plt.xlabel('A_plus')
plt.ylabel('A_minus')
plt.title('Grid search for Aplus,Aminus')